In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import re
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Acquiring preprocessed_dataset
tbs_df = pd.read_csv('/content/drive/My Drive/AAIC Course/Personal case study - StackOverflow/tbs_df.csv')
tbs_df = tbs_df.fillna(' ')

In [ ]:
# defining a function to remove stop_words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.add('would')
stop_words.update([chr(c) for c in range(97, 123)])
# stop_words.remove('no'); stop_words.remove('not'); stop_words.remove('nor')

def stopwrd_removal(sent):
  lst = []
  for wrd in sent.split():
    if wrd not in stop_words:
      lst.append(wrd)
  return " ".join(lst)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def text_preprocessor(column):
  """pass any column with Text in it from tbs_df | Note: returns nothing makes inplace changes in tbs_df"""
  # 1. remove html tags, html urls, replace html comparison operators
  text = tbs_df[column].values
  tbs_df[column] = [re.sub('<.*?>', '', i) for i in text]
  tbs_df[column] = tbs_df[column].str.replace('&lt;', '<')\
                                          .str.replace('&gt;', '>')\
                                          .str.replace('&le;', '<=' )\
                                          .str.replace('&ge;', '>=')

  # 2. remove latex i,e., mostly formulas since it's mathematics based dataset
  tbs_df[column] = [re.sub('\$.*?\$', '', i) for i in text]

  # 3. all lowercase 
  tbs_df[column] = tbs_df[column].str.lower()

  # 4. decontractions
  tbs_df[column] = tbs_df[column].str.replace("won't", "will not").str.replace("can\'t", "can not").str.replace("n\'t", " not").str.replace("\'re", " are").str.\
                                                replace("\'s", " is").str.replace("\'d", " would").str.replace("\'ll", " will").str.\
                                                replace("\'t", " not").str.replace("\'ve", " have").str.replace("\'m", " am")

  # 5. remove all special-characters other than alpha-numericals
  tbs_df[column] = [re.sub('\W', ' ', i) for i in text]
  # # remove all digits
  # tbs_df[column] = [re.sub('\d', ' ', i) for i in text]

  # 6. Stop_word removal
  # tbs_df[column] = [stopwrd_removal(i) for i in text]

  # 7. remove all white-space i.e., \n, \t, and extra_spaces
  tbs_df[column] = [re.sub('  +', ' ', i) for i in text]
  tbs_df[column] = tbs_df[column].str.replace("\n", " ").str.replace("\t", " ").str.strip()
  

In [ ]:
# 1. train_test split

tbs_df['Title_1'] = tbs_df['Title']
text_preprocessor('Title_1')
title = tbs_df['Title_1'].values

# title = tbs_df['Title'].values
# body = tbs_df['Body'].values

# tbs_df['combined_text'] = [t + ' ' + b for t, b in zip(title, body)]
# text_preprocessor('combined_text')
# title_body = tbs_df['combined_text'].values

# # truncating words to 50
# title_body = [' '.join(i.split(' ')[:50]) for i in title_body]

# # # 4. splitting 'combined_text'
# # title = tbs_df['combined_text'].values
# len(title_body)

In [ ]:
final_train = title
final_train[:5], tbs_df['Title'].values[:5]

(array(['euler cycles in biconnected components',
        'can two neighbors in a graph be at the same depth in a dfs tree',
        'unique path in a directed graph',
        'what do you call an average that does not include outliers',
        'correcting for outliers in a running average'], dtype=object),
 array(['Euler cycles in biconnected components',
        'Can two neighbors in a graph be at the same depth in a DFS tree?',
        'Unique path in a directed graph',
        'What do you call an average that does not include outliers?',
        'Correcting for outliers in a running average'], dtype=object))

# 1. USE Embeddings

In [ ]:
import tensorflow as tf

import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Softmax, GRU, LSTM, RNN, Embedding, Dense, RepeatVector, TimeDistributed, Bidirectional, Concatenate
from tensorflow.keras.models import Model

In [ ]:
# loading USE model
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [ ]:
use_embeddings = np.zeros((len(final_train), 512))
batch_size = 128
for i in tqdm(range(int(len(final_train)/batch_size)+1)):
  use_embeddings[i*batch_size : (i+1)*batch_size] = use_model(final_train[i*batch_size : (i+1)*batch_size]).numpy()

In [ ]:
np.save('/content/drive/My Drive/AAIC Course/Personal case study - StackOverflow/use_embeddings.npy', use_embeddings)